In [7]:
from cpe_dict_loader import *
from registry_query import *
from cpe_similarity import *

In [8]:
software_inventory = get_installed_software()

In [9]:
cpe_list = load_cpe_processed_json()

In [10]:
test_set = [{"Vendor":"Google Corporation LTD","Name":"Chrome 0.1.38","Version":"1.54345.123"},
            {"Vendor":"Random Corp","Name":"Discord Chat and Message 0.0.4","Version":"0.5"},
            {"Vendor":"Microsoft Corporation","Name":"Microsoft Edge","Version":"80.0"},
            {"Vendor":"Spotify","Name":"Spotify for Windows","Version":"2"},
            {"Vendor":"Python Software Foundation","Name":"Python","Version":"2"}]

In [11]:
all_matches = []

for software in test_set:
    best_match = {"CPE":None,"Similarity":0}

    for cpe in cpe_list:
        if software.get("Vendor") and software.get("Name"):               
            similarity_score = return_similarity(cpe[1],software.get("Vendor"),software.get("Name"),software.get("Version"))
        
        else: continue

        if similarity_score > best_match["Similarity"] and cpe[1][2] == "a":
            best_match["CPE"] = cpe[0]
            best_match["Similarity"] = similarity_score

    all_matches.append({"Software":software,"Best_match":best_match})

    print(all_matches[-1])


{'Software': {'Vendor': 'Google Corporation LTD', 'Name': 'Chrome 0.1.38', 'Version': '1.54345.123'}, 'Best_match': {'CPE': 'cpe:2.3:a:google:chrome:0.1.38.1:*:*:*:*:*:*:*', 'Similarity': 0.8411308951190849}}
{'Software': {'Vendor': 'Random Corp', 'Name': 'Discord Chat and Message 0.0.4', 'Version': '0.5'}, 'Best_match': {'CPE': 'cpe:2.3:a:discord:discord-recon:0.0.4:*:*:*:*:*:*:*', 'Similarity': 0.6466089943284299}}
{'Software': {'Vendor': 'Microsoft Corporation', 'Name': 'Microsoft Edge', 'Version': '80.0'}, 'Best_match': {'CPE': 'cpe:2.3:a:microsoft:edge:80.0.361.48:*:*:*:*:*:*:*', 'Similarity': 0.7879311956428947}}
{'Software': {'Vendor': 'Spotify', 'Name': 'Spotify for Windows', 'Version': '2'}, 'Best_match': {'CPE': 'cpe:2.3:a:spotify:spotify:1.0.69.336:*:*:*:*:*:*:*', 'Similarity': 0.5467381519846138}}
